In [5]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import (
    ModelCheckpoint, 
    EarlyStopping, 
    ReduceLROnPlateau,
    TensorBoard
)

In [6]:
class Config:
    # Dataset Paths
    DATA_DIR = "./dataset"
    CLASSES = ["anorganik", "organik"]  # Sesuai struktur folder
    
    # Model Parameters
    IMG_SIZE = 300  # Lebih besar untuk akurasi tinggi
    BATCH_SIZE = 32
    EPOCHS = 100
    INIT_LR = 1e-4
    
    # Augmentation
    ROTATION_RANGE = 40
    ZOOM_RANGE = 0.3
    WIDTH_SHIFT_RANGE = 0.2
    
    # Output
    MODEL_DIR = "./models"
    LOG_DIR = "./logs"

config = Config()
os.makedirs(config.MODEL_DIR, exist_ok=True)
os.makedirs(config.LOG_DIR, exist_ok=True)

In [7]:
# Calculate class weights (untuk handle imbalance)
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.arange(len(config.CLASSES)),
    y=np.concatenate([np.full(len(os.listdir(f"{config.DATA_DIR}/{cls}")), i) 
               for i, cls in enumerate(config.CLASSES)])
)
class_weights = dict(enumerate(class_weights))

# Advanced Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=config.ROTATION_RANGE,
    zoom_range=config.ZOOM_RANGE,
    width_shift_range=config.WIDTH_SHIFT_RANGE,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8, 1.2],
    validation_split=0.2,
    fill_mode='nearest'
)

# Train Generator
train_generator = train_datagen.flow_from_directory(
    config.DATA_DIR,
    target_size=(config.IMG_SIZE, config.IMG_SIZE),
    batch_size=config.BATCH_SIZE,
    class_mode='binary',
    subset='training',
    classes=config.CLASSES,
    shuffle=True,
    seed=42
)

# Validation Generator (minimal augmentation)
val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

val_generator = val_datagen.flow_from_directory(
    config.DATA_DIR,
    target_size=(config.IMG_SIZE, config.IMG_SIZE),
    batch_size=config.BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    classes=config.CLASSES,
    shuffle=False
)

Found 18052 images belonging to 2 classes.
Found 4512 images belonging to 2 classes.


In [8]:
def create_generators():
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2
    )

    train_generator = train_datagen.flow_from_directory(
        config.DATA_DIR,
        target_size=(config.IMG_SIZE, config.IMG_SIZE),
        batch_size=config.BATCH_SIZE,
        class_mode='binary',
        subset='training',
        classes=config.CLASSES
    )

    val_generator = train_datagen.flow_from_directory(
        config.DATA_DIR,
        target_size=(config.IMG_SIZE, config.IMG_SIZE),
        batch_size=config.BATCH_SIZE,
        class_mode='binary',
        subset='validation',
        classes=config.CLASSES
    )

    print(f"\n🔍 Dataset Summary:")
    print(f"Training samples: {train_generator.samples}")
    print(f"Validation samples: {val_generator.samples}")
    print(f"Class indices: {train_generator.class_indices}")

    return train_generator, val_generator

train_gen, val_gen = create_generators()

Found 18052 images belonging to 2 classes.
Found 4512 images belonging to 2 classes.

🔍 Dataset Summary:
Training samples: 18052
Validation samples: 4512
Class indices: {'anorganik': 0, 'organik': 1}


In [9]:
def build_advanced_model():
    # Gunakan EfficientNet dengan input size lebih besar
    base_model = EfficientNetB3(
        weights='imagenet',
        include_top=False,
        input_shape=(config.IMG_SIZE, config.IMG_SIZE, 3)
    )
    
    # Freeze base model awal
    base_model.trainable = False
    
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dense(512, activation='swish', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        layers.Dropout(0.5),
        layers.BatchNormalization(),
        layers.Dense(256, activation='swish', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')
    ])
    
    # Custom optimizer dengan weight decay
    optimizer = tf.keras.optimizers.AdamW(
        learning_rate=config.INIT_LR,
        weight_decay=1e-5
    )
    
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall')
        ]
    )
    
    return model

model = build_advanced_model()
model.summary()

43941136/43941136 [==============================] - 37s 1us/step


AttributeError: module 'keras.api._v2.keras.optimizers' has no attribute 'AdamW'

In [ ]:
# Progressive Unfreezing Callback
class UnfreezeCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if epoch == 20:  # Unfreeze setelah 20 epoch
            self.model.layers[0].trainable = True
            for layer in self.model.layers[0].layers[:-20]:  # Hanya unfreeze layer atas
                layer.trainable = False
            self.model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5))

# Callbacks
callbacks = [
    ModelCheckpoint(
        filepath=os.path.join(config.MODEL_DIR, 'best_model.h5'),
        monitor='val_accuracy',
        save_best_only=True,
        mode='max'
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7
    ),
    TensorBoard(
        log_dir=config.LOG_DIR
    ),
    UnfreezeCallback()
]

In [ ]:
history = model.fit(
    train_generator,
    epochs=config.EPOCHS,
    validation_data=val_generator,
    callbacks=callbacks,
    class_weight=class_weights,
    verbose=1
)

In [ ]:
# Plot Training History
def plot_history(history):
    plt.figure(figsize=(14, 5))
    
    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train')
    plt.plot(history.history['val_accuracy'], label='Validation')
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend()
    
    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train')
    plt.plot(history.history['val_loss'], label='Validation')
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(os.path.join(config.LOG_DIR, 'training_history.png'))
    plt.show()

plot_history(history)

# Classification Report
val_generator.reset()
y_pred = model.predict(val_generator)
y_pred = (y_pred > 0.5).astype(int)
y_true = val_generator.classes

print("\n📊 Classification Report:")
print(classification_report(y_true, y_pred, target_names=config.CLASSES))

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=config.CLASSES, 
            yticklabels=config.CLASSES)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig(os.path.join(config.LOG_DIR, 'confusion_matrix.png'))
plt.show()


In [ ]:
model.save(os.path.join(config.MODEL_DIR, 'final_model.h5'))
print("✅ Training completed! Model saved.")